<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E8%A7%92%E8%89%B2%E6%95%B0%E6%8D%AE%E7%BB%9F%E8%AE%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日 Chat-Haruhi-Suzumiya

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()
[![Huggingface Gradio](https://img.shields.io/static/v1?label=Demo&message=Huggingface%20Gradio&color=orange)](https://huggingface.co/spaces/silk-road/ChatHaruhi)


这个脚本主要用来统计每个角色的story数量

---

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing，沈骏一，Aria Fei, 米唯实, 吴平宇, 贾曜恺等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了训练数据自动生成对话部分，设计了整体的路线，并撰写报告。

冷子昂负责了每一个阶段的Gradio开发，以及每个部分的功能整合和架构设计。

闫晨曦一开始将李鲁鲁的notebook重构为app.py，参与了WebUI的embedding部分重构整合。

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具。整合了声纹识别。即将继续参加台本工具的开发。

scixing实践了VITS语音，完成了台词对应的语音抽取，实现了第一个版本的声纹分类。

沈骏一实现了使用ChatGLM2 finetune实验

Aria(Yaying Fei)对接了whisper到台本工具。即将继续参加台本工具的开发。

米唯实实现了Chat哆啦A梦的分支版本

吴平宇部署了ChatGLM2的训练程序，并提供了训练的计算资源。

贾曜恺开发了一个Vue前端实现方案

</details>

In [1]:
!git clone https://github.com/LC1332/Chat-Haruhi-Suzumiya

Cloning into 'Chat-Haruhi-Suzumiya'...
remote: Enumerating objects: 11554, done.
remote: Counting objects: 100% (1672/1672), done.
remote: Compressing objects: 100% (1552/1552), done.
remote: Total 11554 (delta 116), reused 1648 (delta 110), pack-reused 9882
Receiving objects: 100% (11554/11554), 158.05 MiB | 15.01 MiB/s, done.
Resolving deltas: 100% (2086/2086), done.
Updating files: 100% (8099/8099), done.


In [2]:
!pip install -q openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [3]:
%cd /content/Chat-Haruhi-Suzumiya/kyon_generator/
!ls

/content/Chat-Haruhi-Suzumiya/kyon_generator
chat2dialogue.py	   data			  synthesis_chat_method_foo.py
ChatGPT_for_generation.py  dialogue2chat.py	  synthesis_chat.py
chatlog2dialogue.ipynb	   dialogue2embedding.py  test_kyon_generator.ipynb
chatlog2dialogue.py	   story2chat.py	  train.py


我们主要希望扫描所有文件夹之后

对所有文件夹运行这样的指令

```
!python story2chat.py -story_folder "/content/Chat-Haruhi-Suzumiya/characters/haruhi/texts" \
                     -output ./output/chat_from_story.jsonl \
                     -role "春日" \
                     -other_names 凉宫 凉宫春日
```

然后统计生成的jsonl的行数，进行累加，

让我们进行第一步

### 文件夹扫描

请为我实现一段python程序

我希望扫描/content/Chat-Haruhi-Suzumiya/characters/下有多少子文件夹

然后打印成['子文件夹1','子文件夹2']的形式

In [4]:
import os

folder = '/content/Chat-Haruhi-Suzumiya/characters/'
subfolders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]

print(subfolders)

['tangshiye', 'weixiaobao', 'Megumi', 'murongfu', 'liyunlong', 'Luna', 'wangduoyu', 'Ron', 'jiumozhi', 'ayaka', 'personality-data', 'Snape', 'haruhi', 'Malfoy', 'xuzhu', 'xiaofeng', 'duanyu', 'Hermione', 'Dumbledore', 'zhongli', 'wanderer', 'wangyuyan', 'qiaofeng', 'hutao', 'raidenShogun', 'yaemiko', 'lilulu', 'yuqian', 'Harry', 'McGonagall']


In [5]:
count = 0

for folder in subfolders:
    count += 1
    if count % 5 == 0:
        print( f"'{folder}'," ,end = '\\\n')
    else:
        print( f"'{folder}'," , end = '')

'tangshiye','weixiaobao','Megumi','murongfu','liyunlong',\
'Luna','wangduoyu','Ron','jiumozhi','ayaka',\
'personality-data','Snape','haruhi','Malfoy','xuzhu',\
'xiaofeng','duanyu','Hermione','Dumbledore','zhongli',\
'wanderer','wangyuyan','qiaofeng','hutao','raidenShogun',\
'yaemiko','lilulu','yuqian','Harry','McGonagall',\


In [6]:
folder_names = ['tangshiye','weixiaobao','murongfu','liyunlong',\
'Luna','wangduoyu','Ron','jiumozhi',\
'Snape','haruhi','Malfoy','xuzhu',\
'xiaofeng','duanyu','Hermione','Dumbledore',\
'wangyuyan','qiaofeng',\
'yuqian','Harry','McGonagall' ]

role_names = ['汤师爷','韦小宝','慕容复','李云龙',\
'Luna','王多鱼','Ron','鸠摩智',\
'Snape','春日','Malfoy','虚竹',\
'萧峰','段誉','Hermione','Dumbledore',\
'王语嫣','乔峰',\
'于谦','Harry','Professor McGonagall' ]

已知

```python
folder_names = ['tangshiye','weixiaobao','murongfu','liyunlong',\
'Luna','wangduoyu','Ron','jiumozhi',\
'Snape','haruhi','Malfoy','xuzhu',\
'xiaofeng','duanyu','Hermione','Dumbledore','zhongli',\
'wanderer','wangyuyan','qiaofeng',\
'yuqian','Harry','McGonagall' ]

role_names = ['汤师爷','韦小宝','慕容复','李云龙',\
'Luna','王多鱼','Ron','鸠摩智',\
'Snape','春日','Malfoy','虚竹',\
'萧峰','段誉','Hermione','Dumbledore','zhongli',\
'wanderer','王多鱼','乔峰',\
'于谦','Harry','Professor McGonagall' ]
```


我希望批量以

```python
python story2chat.py -story_folder "/content/Chat-Haruhi-Suzumiya/characters/<folder_name>/texts" \
                     -output ./output/<folder_name>_from_story.jsonl \
                     -role "<role_name>"
```

的形式调用脚本

请用python为我实现。并且在运行之前，确认output文件夹的存在性，不存在则新建一个。

In [7]:
if not os.path.exists('./output'):
    os.makedirs('./output')

for folder, role in zip(folder_names, role_names):
    cmd = f"""python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/{folder}/texts'\
     -output ./output/{folder}_from_story.jsonl \
     -role '{role}'"""
    print(cmd)
    os.system(cmd)

python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/tangshiye/texts'     -output ./output/tangshiye_from_story.jsonl      -role '汤师爷'
python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/weixiaobao/texts'     -output ./output/weixiaobao_from_story.jsonl      -role '韦小宝'
python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/murongfu/texts'     -output ./output/murongfu_from_story.jsonl      -role '慕容复'
python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/liyunlong/texts'     -output ./output/liyunlong_from_story.jsonl      -role '李云龙'
python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/Luna/texts'     -output ./output/Luna_from_story.jsonl      -role 'Luna'
python story2chat.py -story_folder '/content/Chat-Haruhi-Suzumiya/characters/wangduoyu/texts'     -output ./output/wangduoyu_from_story.jsonl      -role '王多鱼'
python story2chat.py -story_folder '/content/Chat-Haruh

folder_names = ['tangshiye','weixiaobao','murongfu','liyunlong',\
'Luna','wangduoyu','Ron','jiumozhi',\
'Snape','haruhi','Malfoy','xuzhu',\
'xiaofeng','duanyu','Hermione','Dumbledore','zhongli',\
'wanderer','wangyuyan','qiaofeng',\
'yuqian','Harry','McGonagall' ]

我希望进一步统计所有的./output/{folder}_from_story.jsonl各自有多少行，以 "角色名 行数"的形式输出

In [8]:
import os

count_lines = 0

times = 0

for folder, role in zip(folder_names, role_names):
    file = f'./output/{folder}_from_story.jsonl'
    with open(file) as f:
        lines = len(f.readlines())
        count_lines += lines
    times += 1
    if times % 3 == 0:
        print(f'{role} {lines}')
    else:
        print(f'{role} {lines}', end = '\t\t')

print('总共 ' + str(count_lines) + ' 行')

汤师爷 75		韦小宝 2394		慕容复 158
李云龙 931		Luna 73		王多鱼 115
Ron 1423		鸠摩智 86		Snape 216
春日 457		Malfoy 163		虚竹 197
萧峰 246		段誉 504		Hermione 1418
Dumbledore 436		王语嫣 202		乔峰 190
于谦 774		Harry 2867		Professor McGonagall 169
总共 13094 行


临时代码 ，重新抽取切分一下李云龙

用utf-8格式读取/content/liyunlong_merge.txt

通过\n\n split整个文档，

以不同的文件名，txt格式，utf-8编码,保存所有的非空子字符串到/content/output中

然后把/content/output中的所有txt文件打包压缩

保存到/content/liyunlong.zip

请用python为我实现

In [23]:
import os
import zipfile

input_file = '/content/liyunlong_merge.txt'
output_dir = '/content/output'
archive_file = '/content/liyunlong.zip'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(input_file, 'r', encoding='utf-8') as f:
    content = f.read()

parts = content.split('\n\n')
for i, part in enumerate(parts):
    if part.strip() != '':
        filename = os.path.join(output_dir, str(i) + '.txt')
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(part)

with zipfile.ZipFile(archive_file, 'w') as z:
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        z.write(file_path, filename)

print('Done!')

Done!
